In [37]:
using Pkg
include("./P2H_CapacityExpansion.jl")
cd("/cluster/home/danare/git")
Pkg.activate(".")
using .P2H_CapacityExpansion
using DataFrames
using Parameters
using Flux
using Surrogates
using ScikitLearn
using LinearAlgebra, Random, Statistics
using JuMP
using XLSX
using PlotlyJS
using Clustering
using CSV
using Dates
using StatsBase, MultivariateStats

  Activating project at `~/git`


# Read the Data

In [38]:
file = "/cluster/home/danare/git/P2H_CapacityExpansion/results/aggregated_results/500_scenarios.txt"
file = "/cluster/home/danare/git/P2H_CapacityExpansion/results/500_scenarios_V3.txt"

"/cluster/home/danare/git/P2H_CapacityExpansion/results/500_scenarios_V3.txt"

# Split the Data into Training and Test

In [39]:
df_raw = P2H_CapacityExpansion.read_txt_file(file);
df_raw = select(df_raw, Not(:ENS))
X_train, y_train, X_test, y_test = P2H_CapacityExpansion.partitionTrainTest(df_raw, [:Cost, :Generation, :Emission], 0.7)

([34.0 46.0 … 22.0 0.0; 6.0 8.0 … 18.0 0.0; … ; 56.0 9.0 … 23.0 0.0; 0.0 26.0 … 21.0 0.0], [1291.0 43809.0 1.45124993e8; 1918.0 47135.0 1.32575014e8; … ; 5438.0 54206.0 1.33274952e8; 1162.0 43992.0 1.06995688e8], [62.0 519.0 … 68.0 0.0; 12.0 15.0 … 23.0 0.0; … ; 32.0 36.0 … 22.0 0.0; 6.0 16.0 … 20.0 0.0], [580.0 40764.0 0.0; 1624.0 43308.0 1.24961636e8; … ; 1385.0 44664.0 1.52371092e8; 2088.0 46037.0 1.26214918e8])

In [40]:
### scale the data ###
X_train_scaled, μX, σX  = P2H_CapacityExpansion.scaling(X_train)
X_test_scaled = (X_test .- μX) ./ σX
y_train_scaled, μy, σy  = P2H_CapacityExpansion.scaling(y_train)
    
# remove np.nan #
for i in eachindex(X_test_scaled)
    if isnan(X_test_scaled[i])
        X_test_scaled[i] = 0.0
    end
end

In [ ]:
sg = P2H_CapacityExpansion.simple_neural_network_sklearn(X_train_scaled, y_train_scaled, X_test_scaled; hidden_layer=500, max_iter=1000)
ŷ_rescaled = sg.prediction .* σy .+ μy
r2 = P2H_CapacityExpansion.r2_score(y_test, ŷ_rescaled)
r2

In [41]:
x = permutedims(X_train_scaled)                    # shape: features × samples
y = reshape(y_train_scaled, size(y_train_scaled, 2), :)                  # shape: 1 × samples

m = Flux.Chain(
    Flux.Dense(size(X_train_scaled, 2), 1000, relu),
    Flux.Dense(1000, size(y, 1))
)

# track parameters
θ = Flux.params(m)
# select an optimizer
α = 0.001
opt = ADAM(α)


loss(x, y) = sum((m(x) .- y).^2)
#Flux.Losses.mae(m(x), y)
loss_fn(m, x, y) = sum((m(x) .- y).^2) #Flux.mae(m(x), y) 

train_loader = Flux.DataLoader((x, y), batchsize=32, shuffle=true)
opt_state = Flux.setup(opt, m)

for epoch in 1:1000
    # train the model
    for (x_batch, y_batch) in train_loader
        current_loss, grads = Flux.withgradient(m) do m_in_grad
            loss_fn(m_in_grad, x_batch, y_batch)
        end
        Flux.update!(opt_state, m, grads[1])
    end

    # print report
    ŷ = m(permutedims(X_test_scaled))
    ŷ_rescaled = permutedims(ŷ) .* σy .+ μy
    r2 = P2H_CapacityExpansion.r2_score(y_test, ŷ_rescaled)
    
    println("Epoch = $epoch : Training loss = $(loss(x, y)), R2 score : $(r2)")
    #push!(r2_list, r2)
    #push!(losses, loss(x, y))
end 
ŷ = m(permutedims(X_test_scaled))
ŷ_rescaled = permutedims(ŷ) .* σy .+ μy
r2 = P2H_CapacityExpansion.r2_score(y_test, ŷ_rescaled)

Epoch = 1 : Training loss = 3163.8379208432543, R2 score : 0.5509832833045079
Epoch = 2 : Training loss = 3146.9467851370014, R2 score : 0.535571198408062
Epoch = 3 : Training loss = 3129.201504257183, R2 score : 0.542369738697831
Epoch = 4 : Training loss = 3127.2756022193867, R2 score : 0.5802355140022023
Epoch = 5 : Training loss = 3107.7510050194546, R2 score : 0.5359800425746403
Epoch = 6 : Training loss = 3127.8592949620947, R2 score : 0.6385859661249067
Epoch = 7 : Training loss = 3163.8600988771746, R2 score : 0.6117857457033713
Epoch = 8 : Training loss = 3106.8735151875344, R2 score : 0.6194601389876269
Epoch = 9 : Training loss = 3101.845655543839, R2 score : 0.6475607471840359
Epoch = 10 : Training loss = 3095.0118884578724, R2 score : 0.6480378248103981
Epoch = 11 : Training loss = 3074.239939045554, R2 score : 0.6255249053571457
Epoch = 12 : Training loss = 3090.4453180708633, R2 score : 0.6299045433336312
Epoch = 13 : Training loss = 3094.134779166943, R2 score : 0.58354

0.5128168260518953

In [ ]:
# read in the data
config = P2H_CapacityExpansion.read_yaml_file();
data = P2H_CapacityExpansion.load_cep_data(config=config);
ts_data = P2H_CapacityExpansion.load_timeseries_data_full(config=config);
gas_gen = [gen for (gen, props) ∈ config["techs"] if haskey(props, "input") && get(props["input"], "fuel", nothing) == "R_Gas"];

In [ ]:
# Generate synthetic data for linear regression
x = permutedims(X_train_scaled)  
y = reshape(y_train_scaled, size(y_train_scaled, 2), :) 

# define model architecture

model = Flux.Chain(
    Flux.Dense(size(X_train_scaled, 2), 1000, relu),
    Flux.Dense(1000, size(y, 1)),
    softmax
)

# define loss function

# Define the loss function and optimizer
loss(x, y) = sum((model(x) .- y).^2)

# track parameters

ps = Flux.params(model)

opt = ADAM(0.0001)

# train model


epochs = 1000

for epoch in 1:100
    # train model
    Flux.train!(loss, ps, [(x, y)], opt)
    # print report
    train_loss = loss(x, y)

    ŷ = model(permutedims(X_test_scaled))
    ŷ_rescaled = permutedims(ŷ) .* σy .+ μy
    r2 = P2H_CapacityExpansion.r2_score(y_test, ŷ_rescaled)

    println("Epoch = $epoch : Training Loss = $train_loss, R2 score : $(r2)")
end

ŷ = model(permutedims(X_test_scaled))
ŷ_rescaled = permutedims(ŷ) .* σy .+ μy
r2 = P2H_CapacityExpansion.r2_score(y_test, ŷ_rescaled)


In [ ]:
# load input
using Ipopt, MathOptAI, PyCall
config = P2H_CapacityExpansion.read_yaml_file();
data = P2H_CapacityExpansion.load_cep_data(config=config);
ts_data = P2H_CapacityExpansion.load_timeseries_data_full(config=config);

cep = P2H_CapacityExpansion.run_opt(ts_data=ts_data, data=data, config=config, surrogate=true, solver=Ipopt.Optimizer)

optimizer = optimizer_with_attributes(
    Ipopt.Optimizer,
    "max_iter" => 10000,  # or higher
    "tol" => 1e-5,
    "print_level" => 5,
)

set_optimizer(cep.model, optimizer)

@unpack 𝓖, 𝓨, 𝓣, 𝓡, 𝓢, 𝓛, 𝓒 = P2H_CapacityExpansion.get_sets(cep=cep)
data = data.data;

##################### cost optimization #####################

P2H_CapacityExpansion.setup_opt_costs_fix!(cep, config, data,vcat(cep.sets["non_dispatch"], cep.sets["dispatch"], 𝓢, String[s for s in cep.sets["discharging"]], cep.sets["conversion"]))

#JuMP.fix.(cep.model[:COST]["var", :, :], 0; force=true);
@variable(cep.model, COST_VAR[y ∈ 𝓨] ≥ 0);



In [ ]:
techs = [f for f ∈ cep.sets["invest_tech"] if f != "ENS"]

for y ∈ 𝓨, r ∈ 𝓡
    x_vec = [cep.model[:TotalCapacityAnnual][r, g, y] for g ∈ techs]
    prediction, formulation = MathOptAI.add_predictor(cep.model, m, x_vec)
    println(prediction[2])
    #@constraint(cep.model, COST_VAR[y] .>= prediction)
end

In [ ]:
prediction

# Iterate though the Models

In [ ]:
models = Dict(
    "RandomForest" => P2H_CapacityExpansion.random_forest_sklearn,
    "DecisionTree" => P2H_CapacityExpansion.decision_tree_sklearn,
    "LinearRegression" => P2H_CapacityExpansion.linear_regression_sklearn,
    "NeuralNetwork" => P2H_CapacityExpansion.simple_neural_network_sklearn,
    "GaussianProcesses" => P2H_CapacityExpansion.gaussian_process,
    "SVR" => P2H_CapacityExpansion.svr_sklearn,
);

In [ ]:
function kmeans_subset(X, n)
    R = kmeans(Matrix(X)', n) 
    idx = [findfirst(==(i), R.assignments) for i in 1:n]
    return idx
end

In [ ]:
stochastic_models = Set(["RandomForest", "NeuralNetwork", "DecisionTree"]);

In [ ]:
step_size = 100
df_full = DataFrame(Method=String[],  Value=Float64[], Size=Int[]);
n_runs = 3

for n ∈ 100:step_size:size(df_raw)[1] 
    
    # clustering the entire subspace to identify n samples
    idx = kmeans_subset(select(df_raw, Not(:Cost)), n)
    df = df_raw[idx,:]

    ### split the data into test and training ###
    X_train, y_train, X_test, y_test = P2H_CapacityExpansion.partitionTrainTest(df, [:Cost, :Emission, :Generation], 0.8)

    ### scale the data ###
    X_train_scaled, μX, σX  = P2H_CapacityExpansion.scaling(X_train)
    X_test_scaled = (X_test .- μX) ./ σX
    y_train_scaled, μy, σy  = P2H_CapacityExpansion.scaling(y_train)
    
    # remove np.nan #
    for i in eachindex(X_test_scaled)
        if isnan(X_test_scaled[i])
            X_test_scaled[i] = 0.0
        end
    end

    ### train ML model and compute R2 ### 
    for (name, fun) ∈ models
        # determine average for non-deterministic models
        iter = name ∈ stochastic_models ? n_runs : 1
        
        r2 = 0
        for k ∈ 1:iter
            sg = fun(X_train_scaled, y_train_scaled, X_test_scaled)
            ŷ_rescaled = sg.prediction .* σy .+ μy
            r2 += P2H_CapacityExpansion.r2_score(y_test, ŷ_rescaled)

            ### save the model ###
            if n == last(100:step_size:size(df_raw)[1]) && k == iter
                model = sg.model
                # assume model is trained with ScikitLearn.jl
                joblib.dump(model[:model], "$(dir)$(name).pkl")
            end
        end

        ### add to the df ### 
        push!(df_full, (Method = name, Value = r2/iter, Size = size(X_train)[1]))
    end 
end

In [ ]:
using DataFrames
X_train, y_train, X_test, y_test = P2H_CapacityExpansion.partitionTrainTest(df_raw, [:Cost, :Emission, :Generation], 0.8)
Matrix(DataFrames.dropmissing(y_train))


In [ ]:
y_train

In [ ]:
traces = AbstractTrace[]  # Correct type for individual traces
sort!(df_full, [:Method, :Size], rev=true)

for m in unique(df_full.Method)
    subdf = filter(:Method => ==(m), df_full)
    trace = scatter(
        x = subdf.Size,
        y = subdf.Value,
        mode = "lines+markers",
        name = string(m)
    )
    push!(traces, trace)
end

# Define the layout
layout = Layout(
    title = "Value vs Size by Method",
    xaxis_title = "Training Data",
    yaxis_title = "Accuracy"
)

# Create a single Plot from traces and layout
plt = plot(traces, layout)

# Show the plot
display(plt)

## Sub-sampling strategies

In [ ]:
files_list = filter(f -> endswith(f, ".txt"), readdir(dir, join=true));

In [ ]:
step_size = 50
df_full = DataFrame(Method=String[],  Value=Float64[], Size=Int[]);


### ALTERNATIVE 1: RANDOM SAMPLING ### 
for n in 140:step_size:size(df_raw)[1] #unique(ceil(Int, size(df_raw)[1]/n) for n in 140:step_size:size(df_raw)[1])
    df = df_raw[StatsBase.sample(1:nrow(df_raw), n; replace=false), :]
end

### ALTERNATIVE 2: EQUAL SELECTION SAMPLING ### 
for n in unique(ceil(Int, size(df_raw)[1]/n) for n in 140:step_size:size(df_raw)[1])
    df = df_raw[1:n:end, :]
    df = select(df, names(df)[[sum(df[!, col]) != 0 for col in names(df)]])
end

### ALTERNATIVE 3: LHS ### 
for f in files_list
    df = P2H_CapacityExpansion.read_txt_file(f)
end



# Read, normalize and transpose the df

In [ ]:
df_raw = P2H_CapacityExpansion.read_txt_file(file);
df_raw = select(df_raw, Not(:Cost))
m = Matrix(df_raw)
x_norm, μ, σ = P2H_CapacityExpansion.scaling(m)
X = x_norm'

# Derive PCs

In [ ]:
# generate PCA model
model = fit(PCA, X; maxoutdim=3)

In [ ]:
# transpose the data back again 
X_transform = MultivariateStats.transform(model, X)

In [ ]:
df_pca =   DataFrame(permutedims(X_transform), :auto)
df_pca.Cost = df_raw.Cost

In [ ]:
https://www.reddit.com/r/deeplearning/comments/14vnfe8/how_to_decrease_high_loss_values/
https://discourse.julialang.org/t/how-to-efficiently-and-precisely-fit-a-function-with-neural-networks/73726
https://stackoverflow.com/questions/59153248/why-is-my-neural-network-stuck-at-high-loss-value-after-the-first-epochs


In [ ]:
df_full = DataFrame(Method=String[],  Value=Float64[], Size=Int[]);

df = P2H_CapacityExpansion.read_txt_file(file);

### split the data into test and training ###
X_train, y_train, X_test, y_test = P2H_CapacityExpansion.partitionTrainTest(df, :Cost, 0.7)

### scale the data ###
X_train_scaled, μX, σX  = P2H_CapacityExpansion.scaling(X_train)
X_test_scaled = (X_test .- μX) ./ σX

    # remove np.nan #
for i in eachindex(X_test_scaled)
    if isnan(X_test_scaled[i])
        X_test_scaled[i] = 0.0
    end
end
y_train_scaled, μy, σy  = P2H_CapacityExpansion.scaling(y_train)

### train ML model and compute R2 ### 
for (name,fun) ∈ models
    sg = fun(X_train_scaled, y_train_scaled, X_test_scaled)
    ŷ_rescaled = sg.prediction .* σy .+ μy
    r2 = P2H_CapacityExpansion.r2_score(y_test, ŷ_rescaled)

    ### add to the df ### 
    push!(df_full, (Method = name, Value = r2, Size = size(X_train)[1]))
end


In [ ]:
df_full

In [ ]:
df_full = DataFrame(Method=String[],  Value=Float64[], Size=Int[]);

df = df_pca

### split the data into test and training ###
X_train, y_train, X_test, y_test = P2H_CapacityExpansion.partitionTrainTest(df, :Cost, 0.7)

### scale the data ###
X_train_scaled, μX, σX  = P2H_CapacityExpansion.scaling(X_train)
X_test_scaled = (X_test .- μX) ./ σX

# remove np.nan #
for i in eachindex(X_test_scaled)
    if isnan(X_test_scaled[i])
        X_test_scaled[i] = 0.0
    end
end
y_train_scaled, μy, σy  = P2H_CapacityExpansion.scaling(y_train)

### train ML model and compute R2 ### 
for (name,fun) ∈ models
    sg = fun(X_train_scaled, y_train_scaled, X_test_scaled)
    ŷ_rescaled = sg.prediction .* σy .+ μy
    r2 = P2H_CapacityExpansion.r2_score(y_test, ŷ_rescaled)

    ### add to the df ### 
    push!(df_full, (Method = name, Value = r2, Size = size(X_train)[1]))
end

new sampling technique
iterate more often through the neural network

In [ ]:
df_full

In [ ]:
# Step 3: Loop through files with index

# Parameters
techs = setdiff([key for (key, val) ∈ config["techs"] if get(val, "inv", "")  == true], [key for (key, val) ∈ config["techs"] if get(val, "tech_group", "")  == "transmission"] ) 
years = config["year"]
scenarios = 1:length(txt_files)

# Column names
columns = [:scenario, :year, :cost] ∪ Symbol.(techs)

df = DataFrame(;
    :scenario => repeat(scenarios, inner=length(years)),
    :year => repeat(years, outer=length(txt_files)),
    :cost => fill(0.0, length(txt_files)*length(years)),
)

# Add technology columns, initialized to 0.0
for tech in techs
    df[!, Symbol(tech)] = fill(0.0, length(txt_files)*length(years))
end

## fill in the values
for file in txt_files
    lines = readlines(file)
    
    i = parse(Int64, split(split(file, "/")[end], "_")[1])

    # Parse technology capacities
    for line in lines
        if occursin("TotalCapacityAnnual", line)
            g = split(line, ",")[2]   
            if g in techs
                val = parse(Float64, strip(split(line, "=")[2]))
                y = parse(Int64, split(split(line, "]")[1], ",")[end])

                # insert into the dataframe
                idx = findfirst((df.year .== y) .& (df.scenario .== i))
                df[idx, Symbol(g)] = val
            end

        
        elseif occursin("COSTvar", line)
            y = parse(Int64, line[8:12])
            val = parse(Float64, strip(split(line, "=")[2]))
            # insert into the dataframe
            idx = findfirst((df.year .== y) .& (df.scenario .== i))
            df[idx, :cost] = val
        end
    end
end


In [ ]:
df_agg = df[:, Not(:year, :cost, :scenario)]
X = transpose(Matrix(df_agg))

In [ ]:
ub = [1.25, 620, 460, 300, 0.06, 0.93, 20]
lb = [0.75, 420, 260, 100, 0.015, 0.56, 13]


# Number of samples
n = 3

# Latin Hypercube Sampling
scenarios = Surrogates.sample(n,lb,ub, Surrogates.LatinHypercubeSample())

# 2. TRAIN THE MODEL #

In [ ]:
function perclass_splits(y, percent)
    uniq_class = unique(y)
    keep_index = []
    for class in uniq_class
        class_index = findall(y .== class)
        row_index = randsubseq(class_index, percent)
        push!(keep_index, row_index...)
    end
    return keep_index
end

In [ ]:
y = df[!, :cost]

# split data between train and test
Random.seed!(1)
train_index = perclass_splits(y, 0.67)
test_index = setdiff(1:length(y), train_index)

# spit features
X_train = X[:, train_index]
X_test = X[:, test_index]

# split classes
y_train = transpose(Array{Float64}(y[train_index]))
y_test = transpose(Array{Float64}(y[test_index]))

In [ ]:
model = Chain(
    Dense(12, 32, relu),
    Dense(32, 1)  # output: a single float
)

loss(x, y) = Flux.Losses.mse(model(x), y)  # or Flux.Losses.mae

In [ ]:
# track parameters
ps = Flux.params(model)
 # select an optimizer
learning_rate = 0.01
opt = ADAM(learning_rate)


In [ ]:
# train the model
loss_history = []

epochs = 500

for epoch in 1:epochs
    # train the model
    train!(loss, ps, [(X_train, y_train)], opt)
    # print report
    train_loss = loss(X_train, y_train)
    push!(loss_history, train_loss)
    println("Epoch = $epoch : Training loss = $train_loss")
end 

In [ ]:
# Step 1: Create a combined scenario-year column
df_long[!, :scenario_year] = string.(df_long.scenario, "_", df_long.year)

# Step 2: Pivot wide: rows = technology, columns = scenario_year, values = value
df_wide = unstack(df_long, :technology, :scenario_year, :value)

# Show the result as a matrix
X = Matrix(df_wide[:, Not(:technology)])
X = coalesce.(X, 0.0)
#https://medium.com/@mandarangchekar7/a-neural-network-explained-and-implemented-in-julia-1fbfe4aaf0df


In [ ]:
# make predictions
y_hat_raw = model(X_test)

In [ ]:
y_hat = onecold(y_hat_raw) .- 1
y = y_test_raw
mean(y_hat .== y)

In [ ]:
y_test